In [5]:
from sklearn.model_selection import train_test_split
from torch import nn, optim
from sklearn.model_selection import KFold
import pandas as pd
from datetime import timedelta
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np

In [3]:
#Make dataframe for CNN training
dates = pd.read_pickle("by_date.pkl")
dates = pd.DataFrame(dates)
sp500 = pd.read_pickle("sp500.pkl")
sp500 = pd.DataFrame(sp500)
sp500 = sp500.tail(63).copy()
sp500 = sp500.iloc[::-1]
sp500.reset_index(drop=True, inplace = True)
sp500.drop("open", axis=1, inplace = True)
print(sp500)
sp500['close'] = sp500['close'].diff().apply(lambda x: 1 if x >= 0 else 0)
# sp500 = sp500.iloc[1: , :]
# print(sp500)

dates = dates.T 
dates.columns.name = None
dates.columns = dates.columns.map(str)
dates.reset_index(drop=True, inplace = True)
# display(sp500.to_string())
# display(dates.to_string())
sp500.drop(sp500.index[[10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,
                       40,41,42,43,44]], axis=0, inplace=True)
sp500.reset_index(drop=True, inplace = True)
sp500 = sp500.reindex(columns = sp500.columns.tolist() + ['Today','Yesterday', 'twoDaysPrior',
                                                        'threeDaysPrior', 'fourDaysPrior', 'fiveDaysPrior',
                                                        'sixDaysPrior'])
for row in range(35):
    sp500.at[row, 'sixDaysPrior'] = dates.at[0,sp500.at[row,'date'].strftime("%Y-%m-%d")]
    sp500.at[row, 'fiveDaysPrior'] = dates.at[0,(sp500.at[row,'date']+timedelta(days = 1)).strftime("%Y-%m-%d")]
    sp500.at[row, 'fourDaysPrior'] = dates.at[0,(sp500.at[row,'date']+timedelta(days = 2)).strftime("%Y-%m-%d")]
    sp500.at[row, 'threeDaysPrior'] = dates.at[0,(sp500.at[row,'date']+timedelta(days = 3)).strftime("%Y-%m-%d")]
    sp500.at[row, 'twoDaysPrior'] = dates.at[0,(sp500.at[row,'date']+timedelta(days = 4)).strftime("%Y-%m-%d")]
    sp500.at[row, 'Yesterday'] = dates.at[0,(sp500.at[row,'date']+timedelta(days = 5)).strftime("%Y-%m-%d")]
    sp500.at[row, 'Today'] = dates.at[0,(sp500.at[row,'date']+timedelta(days = 6)).strftime("%Y-%m-%d")]
sp500 = sp500.reset_index().drop("index", axis = 1)
print(sp500)

         date    close
0  2020-04-09  2789.82
1  2020-04-13  2761.63
2  2020-04-14  2846.06
3  2020-04-15  2783.36
4  2020-04-16  2799.55
..        ...      ...
58 2020-07-02  3130.01
59 2020-07-06  3179.72
60 2020-07-07  3145.32
61 2020-07-08  3169.94
62 2020-07-09  3152.05

[63 rows x 2 columns]
         date  close     Today  Yesterday  twoDaysPrior  threeDaysPrior  \
0  2020-04-09      0  1.193676   1.181529      1.149408        1.180180   
1  2020-04-13      0  1.193955   1.143198      1.110070        1.145299   
2  2020-04-14      1  1.097561   1.193955      1.143198        1.110070   
3  2020-04-15      0  1.102343   1.097561      1.193955        1.143198   
4  2020-04-16      1  1.162500   1.102343      1.097561        1.193955   
5  2020-04-17      1  1.151930   1.162500      1.102343        1.097561   
6  2020-04-20      0  1.169811   1.244332      1.100756        1.151930   
7  2020-04-21      0  1.131617   1.169811      1.244332        1.100756   
8  2020-04-22      1  1.14

In [7]:
cv = KFold(n_splits=20, random_state=1, shuffle=True)
model = LogisticRegression()
all_X = sp500.drop(["date", "close"], axis = 1)
all_y = sp500["close"]
scores = cross_val_score(model, all_X, all_y, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores))

0.625


In [ ]:
# Failed attempts:

In [33]:
# class Logistic_Regression(torch.nn.Module):
#     def __init__(self):
#         super(Logistic_Regression,self).__init__()
#         self.layer1=torch.nn.Linear(7,32)
#         self.layer2=torch.nn.Linear(32,1)
#     def forward(self,x):
#         y_predicted=self.layer1(x)
#         y_predicted=torch.sigmoid(self.layer2(y_predicted))
#         return y_predicted

In [35]:
# model=Logistic_Regression()

# criterion=torch.nn.MSELoss()
# optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

# number_of_epochs=100
# for epoch in range(number_of_epochs):
#     y_prediction=model(x_train)
# #     print(y_prediction)
# #     print(y_train)
#     loss=criterion(y_prediction,y_train)
# #     print(loss)
#     loss.backward()
#     optimizer.step()
#     if (epoch+1)%10 == 0:
#         print('epoch:', epoch+1,',loss=',loss.item())
#     optimizer.zero_grad()
# #     break;

epoch: 10 ,loss= 0.24235625565052032
epoch: 20 ,loss= 0.2375141680240631
epoch: 30 ,loss= 0.2332400232553482
epoch: 40 ,loss= 0.22945845127105713
epoch: 50 ,loss= 0.22610056400299072
epoch: 60 ,loss= 0.2231055647134781
epoch: 70 ,loss= 0.22042086720466614
epoch: 80 ,loss= 0.21800199151039124
epoch: 90 ,loss= 0.21581126749515533
epoch: 100 ,loss= 0.21381716430187225


0.625


In [100]:
# print(sp500)
# trainset, testset = train_test_split(sp500, test_size=0.2, random_state = 42)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)
# testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=True)
# print(trainset)

    index       date    close     Today  Yesterday  twoDaysPrior  \
12     12 2020-06-01  3055.73  1.321244   1.266026      1.200748   
8       8 2020-04-22  2799.31  1.147368   1.131617      1.169811   
16     16 2020-06-05  3193.93  1.071517   1.207547      1.242350   
9       9 2020-04-23  2797.80  1.141097   1.147368      1.131617   
33     33 2020-07-08  3169.94  1.217054   1.297462      1.287879   
0       0 2020-04-09  2789.82  1.193676   1.181529      1.149408   
4       4 2020-04-16  2799.55  1.162500   1.102343      1.097561   
17     17 2020-06-15  3066.59  1.290030   1.156863      1.157761   
27     27 2020-06-29  3053.24  1.367647   1.248031      1.205195   
5       5 2020-04-17  2874.56  1.151930   1.162500      1.102343   
11     11 2020-05-29  3044.31  1.237956   1.152310      1.093904   
1       1 2020-04-13  2761.63  1.193955   1.143198      1.110070   
2       2 2020-04-14  2846.06  1.097561   1.193955      1.143198   
32     32 2020-07-07  3145.32  1.297462   1.2878

In [87]:
# class CNN(nn.Module):
#     def __init__(self, in_layer, out_layer, kernel_size, stride):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv1d(in_layer, out_layer, kernel_size=kernel_size, stride=stride, padding = 3, bias=True)
#         self.bn = nn.BatchNorm1d(out_layer)
#         self.relu = nn.ReLU()
    
#     def forward(self,x):
#         x = self.conv1(x)
#         x = self.bn(x)
#         out = self.relu(x)
        
#         return out       


In [79]:
# print(trainset.head(5))

         date    close     Today  Yesterday  twoDaysPrior  threeDaysPrior  \
12 2020-06-01  3055.73  1.321244   1.266026      1.200748        1.237956   
8  2020-04-22  2799.31  1.147368   1.131617      1.169811        1.244332   
16 2020-06-05  3193.93  1.071517   1.207547      1.242350        1.247619   
9  2020-04-23  2797.80  1.141097   1.147368      1.131617        1.169811   
33 2020-07-08  3169.94  1.217054   1.297462      1.287879        1.277657   

    fourDaysPrior  fiveDaysPrior  sixDaysPrior  
12       1.152310       1.093904      1.181818  
8        1.100756       1.151930      1.162500  
16       1.321244       1.266026      1.200748  
9        1.244332       1.100756      1.151930  
33       1.315496       1.190639      1.199561  


In [111]:
# opt = torch.optim.Adam(model.parameters(),lr=learning_rate)
# training_epochs = 10
# criterion = nn.CrossEntropyLoss()
# train_y = trainset["close"]
# train_x = trainset.drop(["date", "close", "index"], axis = 1)
# batch_size = 5
# # print(train_x['close'])

# for e in range(training_epochs):
#     train_losses = []
#     permutation = torch.randperm(trainset.shape[0])
# #     print(permutation)
#     for i in range(0,trainset.shape[0], batch_size):
#         indices = permutation[i:i+batch_size]
#         batch_x, batch_y = train_x.iloc[indices], train_y.iloc[indices]
#         inputs, targets = torch.from_numpy(batch_x), torch.from_numpy(batch_y)
#         inputs, targets = inputs.cuda(), targets.cuda()
#         opt.zero_grad()   
#         output = model(inputs, batch_size)

#         loss = criterion(output, targets.long())
#         train_losses.append(loss.item())
#         loss.backward()
#         opt.step()
#     val_losses = []
#     accuracy=0
#     f1score=0
#     print("Epoch: {}/{}...".format(e+1, training_epochs),
#               "Train Loss: {:.4f}...".format(np.mean(train_losses)))

TypeError: expected np.ndarray (got DataFrame)

In [ ]:

# trainset, testset = train_test_split(sp500, test_size=0.2, random_state = 42)
# x_train = trainset.drop(["date", "close"], axis = 1)
# y_train = trainset["close"]
# x_test = testset.drop(["date", "close"], axis = 1)
# y_test = testset["close"]

# scaler=sklearn.preprocessing.StandardScaler()
# x_train=scaler.fit_transform(x_train)
# x_test=scaler.transform(x_test)

# x_train=torch.from_numpy(x_train.astype(np.float32))
# x_test=torch.from_numpy(x_test.astype(np.float32))
# y_train=torch.from_numpy(y_train.to_numpy().astype(np.float32))
# y_test=torch.from_numpy(y_test.to_numpy().astype(np.float32))

# y_train=y_train.view(y_train.shape[0],1)
# y_test=y_test.view(y_test.shape[0],1)
# # train_x = trainset.drop(["date", "close", "index"], axis = 1)